# Classification

This tutorial uses safeds on **titanic passenger data** to predict who will survive and who will not, using sex as a feature for the prediction.


1. Load your data into a `Table`, the data is available under `docs/tutorials/data/titanic.csv`:


In [ ]:
from safeds.data.tabular.containers import Table

titanic = Table.from_csv_file("data/titanic.csv")
#For visualisation purposes we only print out the first 15 rows.
titanic.slice_rows(0,15)

2. Split the titanic dataset into two tables. A training set, that we will use later to implement a training model to predict the survival of passengers, containing 60% of the data, and a testing set containing the rest of the data.
Delete the column `survived` from the test set, to be able to predict it later:

In [ ]:
split_tuple = titanic.split_rows(0.60)

train_table = split_tuple[0]
testing_table = split_tuple[1]

test_table = testing_table.remove_columns(["survived"]).shuffle_rows()

3. Use `OneHotEncoder` to create an encoder, that will be used later to transform the training table.
* We use `OneHotEncoder` to transform non-numerical categorical values into numerical representations with values of zero or one. In this example we will transform the values of the sex column, hence they will be used in the model for predicting the surviving of passengers.
* Use the `fit` function of the `OneHotEncoder` to pass the table and the column names, that will be used as features to predict who will survive to the encoder.
* The names of the column before transformation need to be saved, because `OneHotEncoder` changes the names of the fitted `Column`s:


In [ ]:
from safeds.data.tabular.transformation import OneHotEncoder

old_column_names = train_table.column_names
encoder = OneHotEncoder().fit(train_table, ["sex"])

4. Transform the training table using the fitted encoder, and create a set with the new names of the fitted `Column`s:


In [ ]:
transformed_table = encoder.transform(train_table)
new_column_names = transformed_table.column_names
new_columns= set(new_column_names) - set(old_column_names)

5. Tag the `survived` `Column` as the target variable to be predicted. Use the new names of the fitted `Column`s as features, which will be used to make predictions based on the target variable.

In [ ]:
tagged_train_table= transformed_table.tag_columns("survived", feature_names=[
    *new_columns
])

6. Use `RandomForest` classifier as a model for the classification. Pass the "tagged_titanic" table to the fit function of the model:

In [ ]:
from safeds.ml.classical.classification import RandomForestClassifier

model = RandomForestClassifier()
fitted_model= model.fit(tagged_train_table)

7. Use the fitted random forest model, that we trained on the training dataset to predict the survival rate of passengers in the test dataset.
Transform the test data with `OneHotEncoder` first, to be able to pass it to the predict function, that uses our fitted random forest model for prediction:

In [ ]:
encoder = OneHotEncoder().fit(test_table, ["sex"])
transformed_test_table = encoder.transform(test_table)

predicition = fitted_model.predict(
    transformed_test_table
)
#For visualisation purposes we only print out the first 15 rows.
predicition.slice_rows(0,15)

8. You can test the accuracy of that model with the initial testing_table as follows:

In [ ]:
encoder = OneHotEncoder().fit(test_table, ["sex"])
testing_table = encoder.transform(testing_table)

tagged_test_table= testing_table.tag_columns("survived", feature_names=[
    *new_columns
])
fitted_model.accuracy(tagged_train_table)
